In [58]:
import pandas as pd

data = pd.read_csv('covid.csv')

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466967 entries, 0 to 466966
Data columns (total 61 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   country                                     466967 non-null  object 
 1   date                                        466967 non-null  object 
 2   total_cases                                 455500 non-null  float64
 3   new_cases                                   453216 non-null  float64
 4   new_cases_smoothed                          451996 non-null  float64
 5   total_cases_per_million                     455500 non-null  float64
 6   new_cases_per_million                       453216 non-null  float64
 7   new_cases_smoothed_per_million              451996 non-null  float64
 8   total_deaths                                455500 non-null  float64
 9   new_deaths                                  453845 non-null  float64
 

In [60]:
data = data.drop(columns=['life_expectancy'])

In [61]:
import json

with open('compact.meta.json', 'r') as file:
	metadata = json.load(file)
 
fields = metadata['fields']

In [62]:
col_desc = pd.DataFrame()
col_desc['column'] = data.columns
col_desc['description'] = col_desc['column'].apply(
	lambda x: fields[x]['description'] if x in fields and 'description' in fields[x] else (
			fields[x]['description_short'] if x in fields and 'description_short' in fields[x] else (
			fields[x]['title'] if x in fields and 'title' in fields[x] else 'No description available'
		)
	)
)

col_desc['description'] = col_desc['description'].str.replace('\n', '')

In [63]:
col_desc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   column       60 non-null     object
 1   description  60 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


In [64]:
data['year'] = data['date'].apply(lambda x: x.split('-')[0])
data['month'] = data['date'].apply(lambda x: x.split('-')[1])
data['day'] = data['date'].apply(lambda x: x.split('-')[2])
data = data.drop(columns=['date'])

last_three_columns = data.iloc[:, -3:]
remaining_columns = data.iloc[:, :-3]
data = pd.concat([remaining_columns.iloc[:, :2], last_three_columns, remaining_columns.iloc[:, 2:]], axis=1)

col_desc = pd.concat([col_desc, pd.DataFrame([
	{'column': 'year', 'description': 'Year of the observations.'},
	{'column': 'month', 'description': 'Month of the observations.'},
	{'column': 'day', 'description': 'Day of the observations.'}
])], ignore_index=True)

col_desc = col_desc[col_desc['column'] != 'date']

In [65]:
# Move the last 3 rows to be the 3rd, 4th, and 5th rows
last_three_rows = col_desc.iloc[-3:]
remaining_rows = col_desc.iloc[:-3]
col_desc = pd.concat([remaining_rows.iloc[:2], last_three_rows, remaining_rows.iloc[2:]]).reset_index(drop=True)
col_desc

,column,description
0,country,Region name.
1,total_cases,Total confirmed cases of COVID-19. Counts can ...
2,year,Year of the observations.
3,month,Month of the observations.
4,day,Day of the observations.
...,...,...
57,extreme_poverty,Percentage of population living in households ...
58,diabetes_prevalence,Diabetes prevalence (% of population ages 20 t...
59,handwashing_facilities,Share of the population with access to a handw...
60,hospital_beds_per_thousand,"Hospital beds (per 1,000 people)"


In [66]:
col_desc.to_csv("columns_description.csv", index=False)

In [67]:
data.to_csv("covid.csv", index=False)